In [1]:
import os
import json
import pandas as pd

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from IPython.core.display import display, HTML

/var/folders/p8/sq7kwdmj7c74xqn10jk560xh0000gn/T/ipykernel_69975/2627644872.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [6]:
_ = load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY')
)

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello"}
    ]
)

print(completion.choices[0].message)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [11]:
data_path = "/Users/dell/Yoav/nlp_project/data/FLenQA.jsonl"

In [12]:
# Initialize an empty list to store each JSON object
data = []

# Open the file and read each line
with open(data_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Parse each line as JSON and append it to the list
        data.append(json.loads(line))

In [29]:
data[9]

{'sample_id': 0,
 'label': 'True',
 'facts': ["John's living room is marble-floored, a reality that is as intrinsic to the building as its very foundations. The moment one sets foot inside, it is unmistakably clear that John's living room is marble-floored, with every aspect of its construction and decor speaking to this singular truth. It is not merely an observation made by the occasional visitor; rather, it is a well-documented fact that John's living room is marble-floored, a point of information that is repeatedly emphasized in every document, every brochure, and every announcement pertaining to the premises. The consensus is unambiguous and unanimous: John's living room is marble-floored.",
  "Ethan Washington is in John's living room, a fact that has become as much a part of the place as the walls and the ceiling. The truth that Ethan Washington is in John's living room is so well-established that it is almost redundant to mention it, yet it is mentioned, again and again, a test

In [30]:
display(HTML(data[9]['mixin'].replace('\n', '<br>')))